In [134]:
def get_surrounding_verses(df, verse_id, window_size):
    all_verses = list(df['ids'].values)
    all_text = list(df['Text'].values)
    if verse_id not in all_verses:
        return []
    
    verse_index = all_verses.index(verse_id)
    surrounding_verses = []
    
    start = max(0, verse_index - window_size)
    end = min(len(all_verses), verse_index + window_size + 1)
    
    surrounding_text = all_text[start:end]
    verse_range = f"{all_verses[start]}-{all_verses[end]}"
        
    return (verse_range,surrounding_text)

def merge(list1, list2):
    res = []
    for i in range(len(list2[0])):
            res.append((list1[0][i],list2[0][i]))
   
     
    return res


def make_verses(df,query):
    res = []
    for verse in query['ids'][0]:
        verse,text = get_surrounding_verses(df,verse,5)
        res.append((verse,text))
        
    return res
        
    

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']    

In [135]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa


# Read the parquet file
table = pq.read_table('christianity_kjv_verses.parquet')

# Convert to pandas DataFrame for easier manipulation
df = table.to_pandas()

# Create a new 'ids' column
df['ids'] = df['Book'].astype(str) + ':' + df['Chapter'].astype(str) + ':' + df['Verse'].astype(str)


In [145]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="~/chroma" # Optional, defaults to .chromadb/ in the current directory
))
collection = client.create_collection(name="kjbv")
verse = list(df['Text'])
embedding = list(df['Embedding'])
ids = list(df['ids'])


collection.add(documents=verse,embeddings=embed,ids=ids)

No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [146]:
import anthropic
import os
import openai


question = "What does it mean to pray?"
query = collection.query(    
    query_embeddings=get_embedding(question),
    n_results=10,
)

prompt = f'''System: You are a wise AI agent that has been ordained to interpret spirtual scripture and engage in dialetic dialogues with the goal of aiding the user and humandkind.
when a user asks a question the system will provide the most related pieces of scripture as context. The user does not have this so do not assume they do and please quote directly. You are to quote those and build a powerful narrative to help enlighten the user. Make sure to include the text quoted for important insights.
Remember to stay in character as a wise guru enlightening the user who is a serious student of philosphy.
verses: {make_verses(df,query)}
question: {question}
Dialogue:'''

c = anthropic.Client(os.environ["ANTHROPIC_API_KEY"])
response = c.completion(
                prompt=f"{anthropic.HUMAN_PROMPT}{prompt}/n{anthropic.AI_PROMPT}",
                stop_sequences=[],
                model='claude-2.0',
                max_tokens_to_sample=10000)

response['completion']

' Here are some relevant passages about prayer from the scripture verses provided:\n\n"Pray without ceasing." (1 Thessalonians 5:17) \n\nThis suggests we should be constantly in an attitude and spirit of prayer, communing with God regularly throughout our day.\n\n"And he said unto them, When ye pray, say, Our Father which art in heaven..." (Luke 11:2)\n\nThis shows that prayer is directed to God our Father in heaven, with reverence and humility. \n\n"And he spake a parable unto them to this end, that men ought always to pray, and not to faint;" (Luke 18:1) \n\nThis parable encourages persistence and perseverance in prayer, not losing heart. \n\n"And Jesus said unto them, Because of your unbelief: for verily I say unto you, If ye have faith as a grain of mustard seed, ye shall say unto this mountain, Remove hence to yonder place; and it shall remove; and nothing shall be impossible unto you. Howbeit this kind goeth not out but by prayer and fasting." (Matthew 17:20-21)\n\nHere Jesus con